In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datasets import load_dataset

HF_USERNAME = "LukeGPT88"
PROJECT_NAME = "patient-doctor-text-classifier"
SUB_PROJECT_NAME = "eng"
DATASET_NAME = f"{HF_USERNAME}/{PROJECT_NAME}-{SUB_PROJECT_NAME}-dataset"

dataset = load_dataset(DATASET_NAME)

In [ ]:
dataset

In [ ]:
len(dataset['train']['text'])

## MEMORY SIZE FOR EACH STRING

In [ ]:
import sys

size_in_bytes = []

for text in dataset['train']['text']:
  size_in_bytes.append(sys.getsizeof(text) - sys.getsizeof(""))

print(f"Total Size of text strings: {sum(size_in_bytes)} bytes")
print(f"Average Size for each string: {sum(size_in_bytes)/len(dataset['train']['text'])} bytes")

## TRANSLATION TASK

In [ ]:
from deep_translator import GoogleTranslator

texts = dataset['train']['text']
res = GoogleTranslator(source='auto', target='it').translate_batch(texts)

In [ ]:
from deep_translator import GoogleTranslator

data = {}
data['en'] = dataset['train']['text']
for lang in ['it']:
  examples_list = []
  for idx, text in enumerate(dataset['train']['text']):
    res = GoogleTranslator(source='auto', target=lang).translate(text)
    examples_list.append(res)
    print(idx)
  data[lang] = examples_list

df = pd.DataFrame(data)
df.to_csv('patient-doctor-text-classifier-it-dataset.csv')